In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
import requests
import csv
import time

In [24]:
# Configure Selenium options
options = Options()
#options.headless = True  # Run in headless mode, without opening a browser window
options.add_argument('-v')  # add the version flag
options.browser_version= '114.0.5735.110'  # set the version to 69.0
options.binary_location = "C:\Program Files\Google\Chrome\Application\chrome.exe" 

# Path to your ChromeDriver executable
driver_path = 'chromedriver.exe'

In [25]:
driver = webdriver.Chrome(service=Service(driver_path), options=options)

In [ ]:
#driver.get("https://www.defensenews.com/industry/techwatch/")

In [26]:
driver.maximize_window()

# Testing and learning
# --------------------------------------

In [ ]:
# Locating the box that contains all the news listed in the page
#Container = driver.find_element(By.XPATH, "//div[@id='c0fC4HMqy1sP5Mn']")
Container = driver.find_element(By.XPATH, "//section[contains(@class,'MainGrid')]")

In [ ]:
Headlines = []

In [ ]:
Headlines_elements = Container.find_elements(By.XPATH,".//h5[@itemprop='headline']|.//h6[@itemprop='headline']")

In [ ]:
for element in Headlines_elements:
    Headlines.append(element.text)
    print(element.text)

In [ ]:
Contents = []

In [ ]:
Contents_elements = Container.find_elements(By.XPATH,".//h5[@itemprop='description']|.//h6[@itemprop='description']")

In [ ]:
for element in Contents_elements:
    Contents.append(element.text)
    print(element.text)

In [ ]:
Images_urls = []

In [ ]:
Images_urls_elements = Container.find_elements(By.XPATH,".//figure[@type='image']/picture/img")

In [ ]:
for element in Images_urls_elements:
    Images_urls.append(element.get_attribute('src'))
    print(element.get_attribute('src'))

In [ ]:
Images_urls

In [ ]:
i = 1
for url in Images_urls:
    response = requests.get(url)
    file = open("image_name{}.jpg".format(i), "wb")
    file.write(response.content)
    file.close()
    i = i + 1

In [ ]:
Title_urls = []

In [ ]:
Title_urls_text = [] # Its basically the headlines :(

In [ ]:
Title_urls_elements = Container.find_elements(By.XPATH,".//h5[@itemprop='headline']/a|.//h6[@itemprop='headline']/a")

In [ ]:
Title_urls_elements[0].get_attribute('href')

In [ ]:
for element in Title_urls_elements:
    Title_urls.append(element.get_attribute('href'))
    Title_urls_text.append(element.text)
    print(element.text)
    print(element.get_attribute('href'))

In [ ]:
#https://www.defensenews.com/industry/techwatch/2023/06/26/heres-who-will-move-forward-in-the-bradley-replacement-competition/
#//figcaption
#//div[@data-gtm-name='Article Body']
#//div[@data-gtm-name='Article Body']/article/p/a
#//h1[contains(@class,'headline')]
#.....................................
#check if the link is not inside the list then append

# End Of Testing
# -----------------------------------

In [5]:
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [42]:
# Set your maximum depth for following links
max_depth = 5

def get_links(Container):
    links = set()
    elements = Container.find_elements(By.XPATH,".//div[@data-gtm-name='Article Body']/article/p/a | .//h5[@itemprop='headline']/a | .//h6[@itemprop='headline']/a")
    for elem in elements:
        link = elem.get_attribute('href')
        if link.startswith('http'):
            links.add(link)
    return links

def get_article_headings(Container):
    final_Headings = list()
    try:
        elements = Container.find_elements(By.XPATH,".//h1[contains(@class,'headline')]")
        for elem in elements:
            text = elem.text
            final_Headings.append(text)
    except:
        pass
    return final_Headings

def get_article_contents(Container):
    article_contents = list()
    try:
        elements = Container.find_elements(By.XPATH,".//div[@data-gtm-name='Article Body']/article/p")
        for elem in elements:
            text = elem.text
            article_contents.append(text)
    except:
        pass
    return article_contents

#def get_headings_links(Container):
#    final_Heading_Links = set()
#    elements = Container.find_elements(By.XPATH,".//h5[@itemprop='headline']/a|.//h6[@itemprop='headline']/a")
#    for elem in elements:
#        link = elem.get_attribute('href')
#        if link.startswith('http'):
#            final_Heading_Links.add(link)
#    return final_Heading_Links

def get_header_contents(Container):
    contents = list()
    try:
        elements = Container.find_elements(By.XPATH,".//h5[@itemprop='description']|.//h6[@itemprop='description']")
        for elem in elements:
            contents.append(elem.text)
    except:
        pass
    return contents

def get_fig_contents(Container):
    fig_contents = list()
    try:
        elements = Container.find_elements(By.XPATH,".//figcaption")
        for elem in elements:
            fig_contents.append(elem.text)
    except:
        pass
    return fig_contents

def get_figure_urls(Container):
    get_figure_urls = set()
    try:
        elements = Container.find_elements(By.XPATH,".//figure[@type='image']/picture/img")
        for elem in elements:
            url = elem.get_attribute('src')
            get_figure_urls.add(url)
    except:
        pass
    return get_figure_urls


In [10]:
visited_links = set()
def crawl_links(driver, url, depth,visited_links,count=0):
    if depth == max_depth:
        return visited_links
    driver.get(url)
    count = count+1
    print("website crawled: ",count)
    print(url)
    with open("output.txt", "a") as f:
        f.write(str(url) + ",")
    time.sleep(4)# Adjust this sleep time based on the website's loading time
    try:
        Container = driver.find_element(By.XPATH, "//section[contains(@class,'MainGrid')]")

        visited_links.add(url)
        links_on_page = get_links(Container)
    except:
        links_on_page = set()

    for link in links_on_page:
        if link not in visited_links:
            visited_links |= crawl_links(driver, link, depth + 1,visited_links,count)

    return visited_links

In [11]:
url = "https://www.defensenews.com/industry/techwatch/"
# Start the crawling process
visited_links = crawl_links(driver, url,0,visited_links)


website crawled:  1
https://www.defensenews.com/industry/techwatch/
website crawled:  2
https://www.defensenews.com/video/2023/07/03/is-the-army-going-back-to-the-classic-three-part-pt-test/
website crawled:  3
https://www.defensenews.com/video/2023/07/03/bae-showcases-recon-amphibious-combat-vehicle-plans-for-recovery-variant/
website crawled:  4
https://www.defensenews.com/global/europe/2023/07/05/britain-seals-typhoon-radar-upgrade-with-11-billion-award/
website crawled:  5
https://www.defensenews.com/battlefield-tech/space/2023/06/28/house-bill-calls-for-test-of-space-force-budget-flexibility/
website crawled:  5
https://www.defensenews.com/unmanned/2023/06/28/autonomous-drones-may-help-air-force-slash-aircraft-inspection-times/
website crawled:  5
https://www.defensenews.com/congress/2023/07/03/lawmaker-summons-pentagon-official-over-canceled-travel-software/
website crawled:  5
https://www.defensenews.com/news/your-marine-corps/2023/07/05/us-military-calls-for-better-weapons-to-f

website crawled:  4
https://www.defensenews.com/global/europe/2023/06/22/setbacks-and-success-two-drone-makers-forge-a-footprint-in-ukraine/
website crawled:  5
https://www.defensenews.com/unmanned/2022/12/23/latvias-atlas-dynamics-to-open-drone-production-plant-in-ukraine/
website crawled:  4
https://www.defensenews.com/video/2019/06/20/take-a-look-at-new-elbit-systems-tech/
website crawled:  4
https://www.defensenews.com/global/europe/2023/06/22/leonardo-to-up-gun-its-m-346-trainer-jet-team-with-airbus-for-sales/
website crawled:  5
https://www.defensenews.com/industry/2023/06/16/leonardos-new-ceo-shakes-up-management/
website crawled:  4
https://www.defensenews.com/unmanned/2023/06/21/leonardo-displays-falco-xplorer-drone-armed-with-mbda-missile/
website crawled:  4
https://www.defensenews.com/video/2019/06/20/heres-how-the-air-force-wants-to-re-engine-b-52-bombers/
website crawled:  4
https://www.defensenews.com/global/europe/2023/06/19/french-army-taps-nexter-to-build-tank-busting

website crawled:  4
https://www.defensenews.com/land/2021/04/19/us-army-picks-5-innovators-to-help-increase-its-howitzer-firing-rate/
website crawled:  5
https://www.defensenews.com/digital-show-dailies/global-force-symposium/2021/03/18/us-army-nears-choosing-first-battalion-for-extended-range-cannon/
website crawled:  4
https://www.defensenews.com/digital-show-dailies/ausa/2019/10/16/army-futures-command-is-leading-a-cultural-shift-much-to-the-delight-of-industry/
website crawled:  3
https://www.defensenews.com/land/2022/02/08/with-new-climate-strategy-army-aims-to-prepare-soldiers-for-harsher-environments/
website crawled:  3
https://www.defensenews.com/land/2022/06/09/lawmakers-want-army-to-set-up-program-to-experiment-with-electrical-tactical-vehicle-operations/
website crawled:  4
https://www.defensenews.com/land/2022/01/25/oshkosh-unveils-hybrid-electric-joint-light-tactical-vehicle/
website crawled:  5
https://www.defensenews.com/land/2019/06/21/after-delay-army-clears-joint-lig

website crawled:  4
https://www.defensenews.com/air/2023/03/07/us-air-force-reworks-tanker-modernization-plans-slashes-next-buy/
website crawled:  4
https://www.defensenews.com/air/2022/03/28/air-force-would-cut-150-aircraft-including-a-10s-buy-fewer-f-35s-in-2023-budget/
website crawled:  5
https://www.defensenews.com/congress/2022/03/23/lawmakers-draw-battle-lines-for-fy23-defense-spending-ahead-of-white-house-budget-release/?msclkid=d2d16ae1aeb911ec9337bbfc6a855495
website crawled:  5
https://www.defensenews.com/congress/budget/2022/03/28/biden-requests-773-billion-for-pentagon-a-4-boost/
website crawled:  5
https://www.defensenews.com/congress/budget/2021/05/28/us-air-force-to-mothball-dozens-of-a-10s-f-15s-and-f-16s-in-fy22-budget/?msclkid=f49fc12eaeb011ec92e8623a0036e653
website crawled:  5
https://www.defensenews.com/congress/budget/2022/03/09/guard-reserve-would-get-20-more-c-130j-transport-aircraft-under-budget-deal/
website crawled:  4
https://www.defensenews.com/air/2023/03/

WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=114.0.5735.199)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x007BA813+48355]
	(No symbol) [0x0074C4B1]
	(No symbol) [0x00655358]
	(No symbol) [0x00647D96]
	(No symbol) [0x0065C37A]
	(No symbol) [0x006ACB1F]
	(No symbol) [0x0069A536]
	(No symbol) [0x006782DC]
	(No symbol) [0x006793DD]
	GetHandleVerifier [0x00A1AABD+2539405]
	GetHandleVerifier [0x00A5A78F+2800735]
	GetHandleVerifier [0x00A5456C+2775612]
	GetHandleVerifier [0x008451E0+616112]
	(No symbol) [0x00755F8C]
	(No symbol) [0x00752328]
	(No symbol) [0x0075240B]
	(No symbol) [0x00744FF7]
	BaseThreadInitThunk [0x762E00C9+25]
	RtlGetAppContainerNamedObjectPath [0x77E57B4E+286]
	RtlGetAppContainerNamedObjectPath [0x77E57B1E+238]


In [12]:

print(len(visited_links))

# Close the webdriver
driver.quit()

205


In [15]:
with open('AllLinksExtracted.txt', 'w') as f:
    for item in visited_links:
        f.write("%s\n" % item)

In [16]:
word = 'defensenews'
urls_2_visit = []
for item in visited_links:
    if word in item:
        urls_2_visit.append(item)

In [17]:
len(urls_2_visit)

185

In [18]:
with open('Links_with_defensenews.txt', 'w') as f:
    for item in urls_2_visit:
        f.write("%s\n" % item)

In [29]:
def extract_links_2_list(elements):
    links = set()
    for elem in elements:
        link = elem.get_attribute('href')
        if link.startswith('http'):
            links.add(link)
            print(link)
    return links        

In [30]:
def extract_texts_2_list(elements):
    texts = list()
    for elem in elements:
        texts.append(elem.text)
        print(elem.text)
    return texts

In [31]:
def save_2_file(name,items):
    with open(name, "a") as f:
        for item in items:
            f.write("%s\n" % item)

In [48]:
def web_crawling(driver,urls):
    article_headings_cleaned = []
    article_contents_cleaned = []
    header_contents_cleaned = []
    fig_contents_cleaned = []
    figure_urls_cleaned = set()
    counting = 0
    for url in urls:
        counting = counting + 1
        driver.get(url)
        time.sleep(4)
        try:
            Container = driver.find_element(By.XPATH, "//section[contains(@class,'MainGrid')]")
            print("first step")
            #----------------------------------------------------------------------------------------
            article_headings = get_article_headings(Container)
            print("2 step")
            #----------------------------------------------------------------------------------------
            article_contents = get_article_contents(Container)
            print("3 step")
            #----------------------------------------------------------------------------------------
            header_contents = get_header_contents(Container)
            print("4 step")
            #----------------------------------------------------------------------------------------
            fig_contents = get_fig_contents(Container)
            print("5 step")
            #----------------------------------------------------------------------------------------
            figure_urls = get_figure_urls(Container)
            print("6 step")
            #------------------------------cleaned data----------------------------------------------
            article_headings_cleaned.extend(article_headings)
            save_2_file("article_headings_cleaned.txt",article_headings)
            print("7 step",article_headings)
            #----------------------------------------------------------------------------------------
            article_contents_cleaned.extend(article_contents)
            save_2_file("article_contents_cleaned.txt",article_contents)
            print("step 8",article_contents)
            #----------------------------------------------------------------------------------------
            header_contents_cleaned.extend(header_contents)
            save_2_file("header_contents_cleaned.txt",header_contents)
            print("step 9",header_contents)
            #----------------------------------------------------------------------------------------
            fig_contents_cleaned.extend(fig_contents)
            save_2_file("fig_contents_cleaned.txt",fig_contents)
            print("step 10",fig_contents)
            #----------------------------------------------------------------------------------------
            figure_urls_cleaned.update(figure_urls)
            save_2_file("figure_urls_cleaned.txt",figure_urls)
            print("step 11",figure_urls)
            #----------------------end---------------------------------------------------------------
            print("end of loop in try number{}".format(counting))
        except Exception as e:
            print('An error occurred: {}'.format(str(e)))
    return figure_urls_cleaned,fig_contents_cleaned,header_contents_cleaned,article_contents_cleaned,article_headings_cleaned

# The Fun Part Of Web Scraping

In [49]:
figure_urls_cleaned,fig_contents_cleaned,header_contents_cleaned,article_contents_cleaned,article_headings_cleaned = web_crawling(driver,urls_2_visit)

first step
2 step
3 step
4 step
5 step
6 step
7 step ['US Army asks Congress to shift dollars to cover long-range cannon munition demo']
step 8 ['WASHINGTON — The Army is asking Congress to shift $24 million in fiscal year 2018 funding to help pay for a demonstration of a hit-to-kill munition critical to its long-range cannon program, according to a reprogramming request dated June 25 and sent to Capitol Hill.', 'The request is part of a larger omnibus reprogramming document seeking permission from congressional defense committees to reallocate funding from the current and two previous fiscal years.', 'The munition is important to the Army’s top priority — Long-Range Precision Fires — because it will be used in the service’s Extended Range Cannon Artillery (ERCA) weapon system under development.', 'The reprogrammed funding will allow the Cannon-Delivered Area Effects Munitions (C-DAEM) program to move into a competitive demonstration phase in FY20, according to Army budget justificatio

first step
2 step
3 step
4 step
5 step
6 step
7 step []
step 8 []
step 9 ['We review “Six Days in Fallujah,” speak with first female commander of a Silent Drill Platoon and see why military charities are getting $100 million.']
step 10 []
step 11 {'https://www.defensenews.com/resizer/aN9OFoF6H_LHwlPtXJLfDBiLZ6g=/800x0/filters:format(jpg):quality(70)/d3k85ws6durfp9.cloudfront.net/06-30-2023/t_29fed750220b4837931056b4932c1cbc_name_WARFIGHTER_LAB_THUMB_1.jpg', 'https://www.defensenews.com/resizer/JifTolENIfHHCSTsclYHE7VFZ_I=/800x0/filters:format(jpg):quality(70)/d3k85ws6durfp9.cloudfront.net/06-23-2023/t_af8ab5a1154a4f1a832e960158d6c485_name_Thumb_DNW_524_Shina_Ship_Visit.jpg', 'https://www.defensenews.com/resizer/dUhfBcaReSPermapQ54B1Tghu1g=/800x0/filters:format(png):quality(70)/d3k85ws6durfp9.cloudfront.net/07-02-2023/t_64939fcef63e45a2bf0a2048a5a05edf_name_acv_development.png', 'https://www.defensenews.com/resizer/9LDMO7Wekk_sOftLGwCh92qaxEM=/800x0/filters:format(jpg):quality(70)/d3k85

first step
2 step
3 step
4 step
5 step
6 step
7 step ['New radar kits abound for the Eurofighter fleet']
An error occurred: 'charmap' codec can't encode character '\u2032' in position 31: character maps to <undefined>
first step
2 step
3 step
4 step
5 step
6 step
7 step []
step 8 []
step 9 ['Author interview with Brett Crozier, former commander of the USS Roosevelt and our congressional reporter shows you how shipbuilding deals are made.']
step 10 []
step 11 {'https://www.defensenews.com/resizer/aN9OFoF6H_LHwlPtXJLfDBiLZ6g=/800x0/filters:format(jpg):quality(70)/d3k85ws6durfp9.cloudfront.net/06-30-2023/t_29fed750220b4837931056b4932c1cbc_name_WARFIGHTER_LAB_THUMB_1.jpg', 'https://www.defensenews.com/resizer/JifTolENIfHHCSTsclYHE7VFZ_I=/800x0/filters:format(jpg):quality(70)/d3k85ws6durfp9.cloudfront.net/06-23-2023/t_af8ab5a1154a4f1a832e960158d6c485_name_Thumb_DNW_524_Shina_Ship_Visit.jpg', 'https://www.defensenews.com/resizer/dUhfBcaReSPermapQ54B1Tghu1g=/800x0/filters:format(png):quality(

first step
2 step
3 step
4 step
5 step
6 step
7 step ['US Marines are developing air-launched swarming munitions for helos']
step 8 ['WASHINGTON — The U.S. Marine Corps intends to replace some decades-old Hellfire missiles with a family of long-range loitering munitions, giving its attack helicopters greater range and lethality for a fight in the Pacific region.', 'This move comes as part of the Corps’ ongoing Force Design 2030 modernization effort to prepare the service to deter or win a fight against China and other potential adversaries.', 'The Marine Corps on Monday released an annual status update on Force Design efforts, which included a nod to the service’s Long-Range Attack Munition effort supported by the undersecretary of defense for research and engineering, “to rapidly develop and field a low-cost, air launched family of loitering, swarming munitions.”', 'In a June 2 call with reporters, Brig. Gen. Stephen Lightfoot, director of the Capabilities Development Directorate, not

3 step
4 step
5 step
6 step
7 step ['Major combat vehicle player won’t play in US Army’s optionally manned fighting vehicle race']
step 8 ['WASHINGTON — BAE Systems has decided it won’t participate in the U.S. Army’s competition to acquire an Optionally Manned Fighting Vehicle, which will ultimately replace the company’s Bradley fighting vehicle in the service’s inventory.', '“After careful consideration we have decided not to move forward with OMFV,” Jim Miller, senior business development director for combat vehicles at BAE told Defense News in an exclusive June 7 interview.', '“I really want to be clear that we are not making a statement about the viability of the OMFV program or the veracity of the requirements or any of that stuff; this is just a business decision we’ve made,” Miller said.', 'The Army issued a request for proposals to competitively build OMFV prototypes, a major effort within its Next-Generation Combat Vehicle portfolio, in March, and there was every expectation t

...

first step
2 step
3 step
4 step
5 step
6 step
7 step []
step 8 []
step 9 ['"A preflight inspection right now can take up to four hours," a Near Earth Autonomy official told reporters. "We can do it in 30 minutes."', "Appropriators expect the move could provide transparency and clarity on how the service's budget would support national security strategy.", 'Under the new arrangement, Lockheed will build a prototype of the Terrestrial Layer System-Echelons Above Brigade at its facility in Syracuse, New York.', 'Leonel Garciga currently serves as the director for information management within the Army’s G-2 intelligence directorate.', 'In an interview, outgoing PEO IEW&S boss Mark Kitz reflected on Army electronic warfare and provided an update on an AI pipeline known as Project Linchpin.']
step 10 ['Then-Air Force Secretary Deborah Lee James, right, greets Sen. John McCain, R-Ariz., before a Senate Armed Services Committee hearing in 2016. (Scott M. Ash/U.S. Air Force)', 'A B-2 Spirit dr

first step
2 step
3 step
4 step
5 step
6 step
7 step ['British military feels out industry for helicopter buy worth over $1 billion']
step 8 ['LONDON — A £1 billion (U.S. $1.25 billion) competition to acquire a new fleet of medium-size helicopters for the British military was officially triggered by the Ministry of Defence this week.', 'The government on May 18 released procurement details to acquire up to 44 helicopters when it gave potential contenders for the contract until June 20 to respond to a pre-qualification questionnaire scoping out their interest.', 'The envisioned New Medium Helicopter “will provide a common medium-lift, multi-role helicopter, fitted for, but not with, specialist mission role equipment and able to operate in all environments in support of defense tasks,” said the industry solicitation', 'The race will see the Airbus H175 M, Lockheed Martin/Sikorsky Blackhawk, Leonardo AW146 and others make a pitch for a program mainly aimed at replacing 23 long-serving Pum

first step
2 step
3 step
4 step
5 step
6 step
7 step ['US Air Force wants to retire all A-10s by 2029']
An error occurred: 'charmap' codec can't encode character '\u2032' in position 104: character maps to <undefined>
first step
2 step
3 step
4 step
5 step
6 step
7 step ['Sweden boosts defense spending, NATO goal in mind']
step 8 ['HELSINKI — Sweden’s newly elected center-right government has prioritized spending on defense-strengthening measures in its draft budget bill for 2023, eying to edge the country closer to NATO’s target quota.', 'Sweden, along with fellow unaligned Nordic nation Finland, is currently awaiting unanimous consent from NATO members to join the alliance, a process that could reach its conclusion in 2023 despite reservations about their applications being expressed by Hungary and Turkey.', 'The “Total Defense” focus in the Swedish government’s defense budgetary plan aims to raise military spending to 2% of gross domestic product by 2026. The defense budget is on co

first step
2 step
3 step
4 step
5 step
6 step
7 step ['Air Force mulls remote control of drone wingmen']
step 8 ['WASHINGTON — The Air Force is studying whether drone wingmen flying alongside piloted fighter aircraft could be controlled by operators in nearby battle management aircraft or refueling tankers, according to Chief of Staff Gen. CQ Brown.', 'The future fleet of collaborative combat aircraft, or CCA, as the service calls the concept, could involve them being partially guided from nearby aircraft such as the KC-46 Pegasus or E-7 Wedgetail, Brown said in a discussion at the Brookings Institution on Monday.', 'The Air Force wants these autonomous CCAs to accompany its future Next Generation Air Dominance fighter, and perhaps also the F-35. Their missions could include striking targets, conducting intelligence, surveillance and reconnaissance missions, or electronic warfare operations such as jamming enemy signals, said Brown.', 'There are a lot of finer points that have to be wo

first step
2 step
3 step
4 step
5 step
6 step
7 step ['F-16 pilot calls the fighter jets sought by Ukraine ‘easy to fly’']
step 8 ['LE BOURGET, France — F-16 fighter jets hurtle pilots through the sky at up to 2,000 kilometers per hour (1,243 miles per hour). In tight turns or sudden climbs, gravity is pressing so hard on their bodies that some of them might even pass out.', 'Even so, it’s an experience Ukraine is seeking for its own pilots after getting U.S. approval to train and eventually receive the American-made jets to bolster its more than year-old fight against Russia. Ukrainian President Volodymyr Zelenskyy had long pressed Western allies for the planes to gain an advantage in the sky.', 'But to be a “Top Gun,” you need more than just the aircraft: You need the training to get you there.', 'Speaking at the Paris Air Show this week, the world’s top aerospace trade show, U.S. Air Force Capt. David “Spicy” Brown, an F-16 pilot, said training takes about a year for American milita

In [50]:
figure_urls_cleaned

{'https://www.defensenews.com/resizer/-o_KD2bQXgu-kedSpW8pDywUIdA=/1024x0/filters:format(jpg):quality(70)/cloudfront-us-east-1.images.arcpublishing.com/archetype/PGNHXKCKQFFBVHCK5TRYCWEHFI.jfif',
 'https://www.defensenews.com/resizer/-s1q0u1thFTl8NsFJlbrisHMITg=/800x0/filters:format(jpg):quality(70)/cloudfront-us-east-1.images.arcpublishing.com/archetype/MB5SA4PLWVHFZKCZ5U4KSTDCRU.jpg',
 'https://www.defensenews.com/resizer/0-t_uyooaYmDmP0hW8Isqrx3CjQ=/1024x0/filters:format(jpg):quality(70)/cloudfront-us-east-1.images.arcpublishing.com/archetype/EIAWW3QOJVBD3HFQUJBPZUBIEQ.jpg',
 'https://www.defensenews.com/resizer/03zmO7m1L1D8jTawOfQ85R_16a8=/1024x0/filters:format(jpg):quality(70)/cloudfront-us-east-1.images.arcpublishing.com/archetype/YKMESXF2DJDFDH66F2KOHOLEZY.jpg',
 'https://www.defensenews.com/resizer/0ugf9Y4s3YDVCIPCPZLmXx6GQFk=/1024x0/filters:format(jpg):quality(70)/cloudfront-us-east-1.images.arcpublishing.com/archetype/VRFLG4G34VECHNUFFOTZF7QQHA.jpg',
 'https://www.defensenews

In [52]:
i=1
for url in figure_urls_cleaned:
    try:
        response = requests.get(url)
        file = open("defensenewsImage{}.jpg".format(i), "wb")
        file.write(response.content)
        file.close()
    except Exception as e:
        print('An error occurred: {}'.format(str(e)))
    i = i + 1
    print(i)

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
An error occurred: HTTPSConnectionPool(host='www.defensenews.com', port=443): Max retries exceeded with url: /resizer/rNNujKn0nPOjTnOBeKzTvnFnMow=/400x0/filters:format(jpg):quality(70)/cloudfront-us-east-1.images.arcpublishing.com/archetype/TIKYXHYAARAA7KH3FWF32OQNFU.jpg (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:1007)')))
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
18